In [73]:
from os import listdir
from os.path import isfile, join
from PIL import Image

import random
import math
import time
import numpy as np
import cv2

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import torch
import torchvision.models.segmentation
import torchvision.transforms as tf
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import DataLoader, Dataset

In [74]:
width = 180
height = 56

In [75]:
class netNvidia(nn.Module):
    def __init__(self):
        super(netNvidia, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3,24,5, stride=2),
            nn.ReLU(),
            nn.Conv2d(24,36,5, stride=2),
            nn.ReLU(),
            nn.Conv2d(36,48, 5, stride=2),
            nn.ReLU(),
            nn.Conv2d(48,64,3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64,64,3, padding=1),
            #nn.Flatten(),
            nn.Dropout(0.5)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=64*4*19, out_features=100),
            nn.ReLU(),
            nn.Linear(in_features=100, out_features=50),
            nn.ReLU(),
            nn.Linear(in_features=50, out_features=10),
            nn.Linear(in_features=10, out_features=1),
        )

    def forward(self, input):
        input = input.view(input.size(0), 3, height, width)
        output = self.conv_layers(input)
        #print(output.shape)
        output = output.view(output.size(0), -1)
        output = self.linear_layers(output)
        return output

In [76]:
model = netNvidia().cuda()
torch.device('cuda')
summary(model, (3, height, width))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 26, 88]           1,824
              ReLU-2           [-1, 24, 26, 88]               0
            Conv2d-3           [-1, 36, 11, 42]          21,636
              ReLU-4           [-1, 36, 11, 42]               0
            Conv2d-5            [-1, 48, 4, 19]          43,248
              ReLU-6            [-1, 48, 4, 19]               0
            Conv2d-7            [-1, 64, 4, 19]          27,712
              ReLU-8            [-1, 64, 4, 19]               0
            Conv2d-9            [-1, 64, 4, 19]          36,928
          Dropout-10            [-1, 64, 4, 19]               0
           Linear-11                  [-1, 100]         486,500
             ReLU-12                  [-1, 100]               0
           Linear-13                   [-1, 50]           5,050
             ReLU-14                   